# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2025-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---

# <h1><center>Clase 03: Trabajo con Datos Estructurados</center></h1>

Este ejercicio busca que los estudiantes usen algunas librerías para importar datos y luego aprendan algunos comandos de Pandas para analizar e inspeccionar los datos.

## 1. Datos para los ejemplos

Usaremos dos conjuntos de datos para este Notebook:
<ol>
<li>Datos de viajes en Taxi en la ciudad de Nueva York:</li>

El proyecto <i>Open Data</i> de la Ciudad de Nueva York nos da acceso a una gran cantiadad de datos del quehacer de la ciudad. En este caso usaremos el sitio con los datos de viajes en Taxi, disponibles en https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
    
En la carpeta '`data`' está disponible la base de datos de todos los viajes realizados en Mayo de 2024. El archivo se llama '`yellow_tripdata_2024-05.parquet`'.

El formato PARQUET, que es open-source desarrollado por Apache, es un formato eficiente para almacenar y leer bases de datos de gran tamaño. Para poder leer este formato desde Python, deberán instalar una nueva librería llamada ' `pyarrow` '. Para instalarla use el comando:

<code>> conda install pyarrow</code>
    
<li>Datos de casos de COVID en Chile:</li>

Durante la pandemia, el Ministerio de Ciencia y Tecnología, con el apoyo de múltiples grupos de investigación y universidades, armó un repositorio abierto de datos sobre la situación de la pandemia en Chile. Los datos a utilizar en este ejemplo provienen del repositorio GitHub **Datos-COVID19** disponible en https://github.com/MinCiencia/Datos-COVID19. Estaremos usando el 

"*Data Product 1 - Casos totales por comuna incremental: el archivo Covid-19.csv contiene las columnas 'Región', ‘Código Región’, 'Comuna', ‘Código comuna’, 'Población', múltiples columnas correspondientes a '[fecha]', y una columna 'Tasa'. Estas últimas columnas, ‘[fecha]’, contienen los 'Casos Confirmados' reportados por el Ministerio de Salud de Chile en cada una de las fechas que se indican en las respectivas columnas.*" https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto1

En la carpeta '`data`' está disponible la base de datos de todos los casos confirmados hasta enero de 2023.
</ol>

## 2. Librerías

El trabajo de esta clase se centrará en el uso de NumPy y Pandas, pero necesitamos otras librerías adicionales para leer los datos y graficar información.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

## 3. Navegación en carpetas y acceso a datos

Para improtar los archvios de datos, necesitamos identificar en qué directorio están guardados en nuestro sistema y en qué directorio estamos trabajando ("working directory").

Algunos comandos importantes:
- `%ls`: lista el contenido del directorio actual command lists all content in the current working directory.
- `%cd 'subdirectorio'`: permite cambiar la ubicación actual a 'subdirectorio'
- `%cd ..`: permite navegar hacia atrás al directorio superior del actual
- `%pwd`: entrega la ruta del directorio actual
      

In [ ]:
%pwd

In [ ]:
%ls

In [ ]:
%cd data

## 4. Estudio de datos de viajes en NYC

El objetivo de este ejercicio es entender cuántos viajes ocurrieron en la ciudad de Nueva York durante mayo de 2023 y cuáles son los lugares más relevantes para tomar pasajeros.

### 4.1 Importar datos

El primer paso será importar los datos, que están en un archivo en formato Parquet, y pasarlos a un DataFrame de Pandas.

In [ ]:
# leer la base parquet
trips = pq.read_table('yellow_tripdata_2024-05.parquet')

### 4.2 `pandas` y DataFrames

El método `<x>.to_pandas()` permite transformar el archivo Parquet en un DataFrame de Pandas.
Un DataFrame es una base de datos estructurada, que posee columnas y filas con la información relevante. Una referencia rápida a varias cosas que se pueden hacer con DataFrames está disponible acá: https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html

In [ ]:
# transformar a dataframe de pandas
trips = trips.to_pandas()

### 4.3 Leer e inspeccionar un DataFrame

Un DataFrame de Pandas posee una serie de métodos que permiten revisar los datos contenidos en el DataFrame. Algunos de los más relevantes los vemos a continuación.

In [ ]:
trips.head()

In [ ]:
trips.info()

In [ ]:
# nombres de las columnas
trips.columns

In [ ]:
# obtener los datos de una columna
trips["trip_distance"]

In [ ]:
# obtener una fila del DataFrame
trips["trip_distance"][1]

### 4.4 Contestando la pregunta

A continuación haremos una serie de cálculos para contestar nuestra pregunta inicial: ¿Cuál es el mejor lugar en NYC para tomar pasajeros?

Cada columna de un DataFrame es una Serie, que corresponde a un arreglo 1-D con una etiqueta. Por lo tanto, en el caso de columnas con datos numéricos, podemos aplicar todas las operaciones matemáticas disponibles en `numpy`: https://numpy.org/doc/stable/reference/routines.math.html

Partamos con la cantidad total de pasajeros que viajaron en Mayo de 2023.

In [ ]:
# suma de la columna de pasajeros
total_pasajeros = trips["passenger_count"].sum()
print(f'La cantidad total de pasajeros fue de {total_pasajeros}')

In [ ]:
# ahora veamos esa suma por cada zona
trips_by_loc = trips[["PULocationID", "passenger_count"]].groupby("PULocationID").sum()
trips_by_loc.head()

In [ ]:
max_loc = trips_by_loc.idxmax()
max_loc

In [ ]:
trips_by_loc.loc[max_loc]

¿Era esperable el resultado?

Acá pueden ver un mapa con el ID de los diferentes lugares:

<img style="float: left; padding-right: 20px; width: 500px" src="https://www.nyc.gov/assets/tlc/images/content/pages/about/taxi_zone_map_queens.jpg">

## 5. Estudio de datos de enfermos COVID

El primer paso es importar los datos estructurados del CSV. Para ello, podemos usar una librería de NumPy que permite importar datos en formato CSV directamente.

### 5.1 `numpy`: np.loadtxt() y np.genfromtxt()

Numpy provee funciones para leer archivos de texto estructurado directamente como arreglos (`np.ndarray`). 

En primer lugar la función `np.loadtxt()`, permite cargar archivos cuyo contenido es solamente numérico. Generalmente trabajaremos con datasets que tienen distintos tipos de datos en distintas columnas; por ejemplo, strings y floats. En este caso, es necesario utilizar la función `np.genfromtxt()`, que puede manejar este tipo de datos. Si usamos como argumento `dtype=None`, la función infiere el tipo de datos de cada columna en forma automática.

La documentación de ambas funciones se encuentra en: <br>
- https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html
- https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html

In [ ]:
# nombre del archivo a leer
data_file='Covid-19.csv'

In [ ]:
# cargar el archivo
data = np.loadtxt(data_file, delimiter=',',dtype='str')#skiprows=1

# Algunas formas de explorar los datos:
print(data[1])
#print(data.shape)
#print(data)

In [ ]:
# Importar data como floats y saltar la primera fila: data_float
data = np.genfromtxt(data_file, delimiter=',', dtype=None, skip_header=1, encoding=None)

print(data[0])
print(data.shape)

#numpy se las puede arreglar con datos mezclados pero es mejor pandas

En general, `numpy` hace un buen trabajo identificando los tipos en conjuntos de datos con tipos mezclados, pero la librería natural para trabajar con datos estructurados es `pandas`.

### 5.2 `pandas`: read_csv para pasar a DataFrame

La función `pd.read_csv()` permite leer un archivo de texto en formato CSV (comma separated value) y generar un DataFrame.
El delimitador por defecto es la coma (,), pero también pueden leerse datasets con otros tipos de separación, especificando el parámetro `delimiter`.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [ ]:
# leer los datos
data = pd.read_csv(data_file, delimiter=',')
data.head()

In [ ]:
data.columns

In [ ]:
type(data['Region'])

In [ ]:
data.tail(5)

In [ ]:
data.info()

### 5.2 Índices y acceso a información de celdas

Al igual que en el caso de NYC, podemos acceder a datos específicos de la base en forma simple.

In [ ]:
# resumen de datos por comuna para un día en particular
datos_resumen = data[['Comuna','26-12-2022']]
datos_resumen

In [ ]:
# Columna de un día en particular
data['26-12-2022']

In [ ]:
# nombre de las comunas
data['Comuna']

In [ ]:
# filtrar los datos para una comuna en particular
data_macul = data[data['Comuna']=='Macul']
data_macul

In [ ]:
# identificar las fechas disponibles
columnas = data.columns[5:-1]
columnas

In [ ]:
# sólo los datos de las columnas con fecha
datos_enfermos = data[columnas]
datos_enfermos

In [ ]:
# información estadística por cada columna
data.describe()

### 5.3 Operaciones con columnas

Cada columna de un DataFrame es una Serie, que corresponde a un arreglo 1-D con una etiqueta. Por lo tanto, en el caso de columnas con datos numéricos, podemos aplicar todas las operaciones matemáticas disponibles en `numpy`:

https://numpy.org/doc/stable/reference/routines.math.html

También podemos realizar operaciones entre 2 o más columnas, o entre columnas y escalares.


In [ ]:
# calcular la cantidad de casos por region
enfermos_por_region = data.groupby("Region").sum(numeric_only=True)
# mostrar sólo los datos de enfermos
enfermos_por_region[columnas]

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(enfermos_por_region.loc['Maule'][columnas], 'x-', label='Maule')
plt.plot(enfermos_por_region.loc['Magallanes'][columnas], 'x-', label='Magallanes')
plt.legend()
plt.xlabel('Fecha')
plt.ylabel('Casos Acumulados')
plt.show()